In [32]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [33]:
img = cv2.imread('IMG_20181014_193248.jpg')

img = cv2.resize(img,(950,950))

In [34]:
img_gray = cv2.imread('IMG_20181014_193248.jpg',0)

img_gray = cv2.resize(img_gray,(950,950))

In [69]:
cv2.imshow('img',img)
cv2.imshow('img_gray',img_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [36]:
def auto_canny(image,sigma=0.33):
    #compute the median of the single channel pixel intensities
    v = np.median(image)
    
    #apply automatic canny edge detection using the computed median
    lower = int(max(0,(1.0-sigma)*v))
    upper = int(min(255,(1.0+sigma)*v))
    edged = cv2.Canny(image,lower,upper)
    
    #return the image
    return edged

In [37]:
img_gray.shape

(950, 950)

In [39]:
img_blur = cv2.GaussianBlur(img_gray,(5,5),0)

wide = cv2.Canny(img_blur,10,200)
tight = cv2.Canny(img_blur,200,225)
auto = auto_canny(img_blur)

cv2.imshow('img_blur',img_blur)
cv2.imshow('edges',np.hstack([wide,tight,auto]))    
cv2.waitKey(0)
cv2.destroyAllWindows()

In [40]:
image, contours, hierarchy = cv2.findContours(auto,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [42]:
#sorting contours (by contour area)
contours = sorted(contours,key=cv2.contourArea,reverse=True)

In [43]:
contours

[array([[[307,  36]],
 
        [[306,  37]],
 
        [[305,  37]],
 
        ...,
 
        [[313,  37]],
 
        [[311,  37]],
 
        [[310,  36]]], dtype=int32), array([[[307,  36]],
 
        [[310,  36]],
 
        [[311,  37]],
 
        ...,
 
        [[299,  45]],
 
        [[301,  43]],
 
        [[301,  42]]], dtype=int32), array([[[496, 255]],
 
        [[496, 258]],
 
        [[497, 259]],
 
        [[497, 260]],
 
        [[498, 260]],
 
        [[500, 258]],
 
        [[500, 257]],
 
        [[498, 255]]], dtype=int32), array([[[794,   1]],
 
        [[793,   2]],
 
        [[792,   2]],
 
        [[791,   3]],
 
        [[790,   3]],
 
        [[789,   4]],
 
        [[788,   4]],
 
        [[787,   5]],
 
        [[786,   5]],
 
        [[785,   6]],
 
        [[784,   6]],
 
        [[783,   7]],
 
        [[782,   7]],
 
        [[780,   9]],
 
        [[779,   9]],
 
        [[778,  10]],
 
        [[777,  10]],
 
        [[775,  12]],
 
        [[774,  12]],


In [44]:
doc_cnt = []
for c in contours:
    peri = cv2.arcLength(c,True)
    approx = cv2.approxPolyDP(c,0.02*peri,True)
    
    if len(approx) == 4:
        doc_cnt.append(approx)
        break

In [45]:
doc_cnt = np.array(doc_cnt)

In [46]:
len(doc_cnt)

1

In [47]:
approx

array([[[305,  37]],

       [[ 49, 356]],

       [[652, 636]],

       [[917, 309]]], dtype=int32)

In [58]:
type(doc_cnt)

numpy.ndarray

In [61]:
type(approx)

numpy.ndarray

In [62]:
doc_cnt

array([[[[305,  37]],

        [[ 49, 356]],

        [[652, 636]],

        [[917, 309]]]], dtype=int32)

In [49]:
img_cnt = cv2.drawContours(img, doc_cnt, 0, (255,255,0), 3)

cv2.imshow('contour image',img_cnt)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [63]:
pts1 = np.float32(approx)
pts2 = np.float32([[950,0],[0,0],[0,950],[950,950]])

In [64]:
M = cv2.getPerspectiveTransform(pts1,pts2)

In [65]:
dst = cv2.warpPerspective(img_gray,M,(900,900))

In [66]:
cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [70]:
#gaus = cv2.adaptiveThreshold(dst, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 120, 1)
ret,thresh = cv2.threshold(dst,110,255,cv2.THRESH_TOZERO)

cv2.imshow('thresh',thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()
#plt.imshow(gaus)